# Import Library

In [16]:
from twython import Twython
import sqlite3
from time import sleep
import numpy as np
import pandas as pd
import pickle
import warnings
from util import JSONParser
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re
import string
import math
import random
warnings.simplefilter(action='ignore', category=FutureWarning)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import warnings
warnings.filterwarnings('ignore') 

# Access API

In [15]:
import tweepy

# Authenticate to Twitter
auth = tweepy.OAuthHandler("CkhGa0z1CKMlK7GY56Zxh0250", 
    "Iq1o1IwRzuFw6xCandQbGUX1gzx6GezPD7ek1ptbcw46eni3LB")
auth.set_access_token("492948056-n4H0mlG3efeuJnDKkkoyeW9BWbl7kBrI95Dmlup1", 
    "jMSNiTz1mo764Xnz1x8cEAuu2CQRIhRbstAgRiD5WY7cP")

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


## Connection Database

In [3]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

In [4]:
con = create_connection("twit copy.db")
cursor=con.cursor()

# import json parser

In [5]:
jp=JSONParser()
jp.parse("data/intents.json")

# Important Function

## open model pkl

In [17]:
def open_model(model_path):
    with open (model_path, 'rb') as f:
        model = pickle.load(f)
    return model
model=open_model("model/model_2.pkl")

## Document Preprocessing

In [7]:
def document_processing(document):
    document = document.lower()
    document = document.translate(str.maketrans("","",string.punctuation))
    document = re.sub("[^A-Za-z\s']"," ", document)
    document = re.sub(r'http\S+', '', document) # remove links
    document = re.sub(r"www.\S+", " ", document) # remove link
    document = document.strip()
    stemmer = StemmerFactory().create_stemmer()
    document = stemmer.stem(document)
    return document

## get response

In [8]:
def generateOTP() :
    digits = "0123456789"
    OTP = ""
    for i in range(5) :
        OTP += digits[math.floor(random.random() * 10)]
    return OTP

In [9]:
def response_trouble(teks, phase):
    print(phase)
    if phase==0:
        if teks == '#gantiATM':
            response= """Baik kak aku bantu proses pergantian ATM ya, Sebelumnya ada beberapa syarat yang harus dilengkapi yaitu dengan isi data diri. Pertama silahkan ketikkan data dengan format *Nama, Nomor Rekening, No Handphone, Email*
            """
            return 1, response, 1
        else:
            proba=model.predict_proba([teks])[0]
            if proba.max()<0.25: response="Maaf Kak Saya Tidak Mengerti(from response trouble)"
            else:
                tag=model.predict([teks])[0]
                response=jp.get_response(tag)
                return 0, response, None
    elif phase==1:
        data_nasabah=teks.split(',')
        data_nasabah=[dt.strip() for dt in data_nasabah]
        try:
            print(data_nasabah)
            query=f"select * from nasabah where no_rekening='{data_nasabah[1]}' and nama= '{data_nasabah[0]}' and no_hp='{data_nasabah[2]}' and email='{data_nasabah[3]}'"
            print(query)
            data_f_sql=cursor.execute(query).fetchall()[0]
            otp=generateOTP()
            query_2=f"UPDATE nasabah set otp={otp} WHERE no_rekening='{data_nasabah[1]}'"
            cursor.execute(query_2)
            con.commit()
            response="Kami telah mengirim kode OTP melalui no HP Anda. Silakan balas pesan ini dengan format *no rekening, otp*"
            return 1, response, 2
        except Exception as error:
            print(error, 'response_trouble')
            response='data yang anda masukkan salah, silahkan masukkaan kembali data anda'
            return 1, response, 1
    elif phase==2:
        data_nasabah=teks.split(',')
        data_nasabah=[dt.strip() for dt in data_nasabah]
        try:
            query=f"SELECT * FROM nasabah WHERE no_rekening = '{data_nasabah[0]}' AND otp={data_nasabah[1]}"
            data_f_sql=cursor.execute(query).fetchall()[0]
            response="Selamat kakak akan menerima kartu ATM baru! Tunggu kami ya kak maksimal 3x24 jam kartu ATM baru kakak sampai rumah"
            query_2=f"UPDATE nasabah set otp=null WHERE no_rekening='{data_nasabah[0]}'"
            cursor.execute(query_2)
            con.commit()
            return 0, response, None
        except Exception as error:
            print(error,  'response_trouble')
            response='data yang anda masukkan salah(from response trouble)'
            return 1, response, 2

        

In [10]:
def response(data):
    data.iloc[:]['text']=data['text'].apply(document_processing)
    resp=[]
    for d in data.values:
        try:
            query=f"""SELECT is_trouble, phase FROM direct_massage 
                                        WHERE 
                                            sender={d[1]} AND
                                            is_trouble=1 AND
                                            ID=(SELECT MAX(ID) FROM direct_massage WHERE sender={d[1]})"""
            q_result=cursor.execute(query).fetchall()[0]
            t, r, phase=response_trouble(d[0],q_result[1])
            resp.append([r, t, phase])
        except Exception as error:
            print(error, '-response')
            proba=model.predict_proba([d[0]])[0]
            if proba.max()<0.25: resp.append(["Maaf Kak Saya Tidak Mengerti(from respons)", False, None])
            else:
                tag=model.predict([d[0]])[0]
                if tag=='ganti ATM':t=1
                else: t= 0
                r=jp.get_response(tag)
                resp.append([r, t, 0])
    print(resp)
    return resp

## grab massage

In [11]:
def get_massage(api):
    pesan=api.get_direct_messages()
    df= pd.DataFrame([[int(s.id), s.created_timestamp, s.message_create['message_data']['text'], s.message_create['target']['recipient_id'], s.message_create['sender_id']] for s in pesan], columns=('ID', 'created at', 'text', 'recipient', 'sender' ))
    return df[df.sender!='492948056']

## check new massage

In [12]:
def cek_response(api, cursor):
    max_id=cursor.execute('SELECT MAX(ID) FROM direct_massage').fetchall()[0][0]
    if max_id==None:max_id=0
    new_massage=get_massage(api)
    if new_massage.ID.max()>max_id:
        massage=new_massage[new_massage.ID>max_id]
        massage[['response','is_trouble', 'phase']]=response(massage[['text','sender']])
        for mas in massage['ID'].values:
            # print(massage[massage.ID==mas].response.values[0])
            try:
                api.send_direct_message(massage[massage.ID==mas].sender.values[0], massage[massage.ID==mas].response.values[0])
            except Exception as error:
                print(error, '->cek response')
        return True, massage
    else:
        return False, None

# Main Menu

## run bot

In [13]:
while True:    
    got, data=cek_response(api, cursor)
    if got: 
        data.to_sql('direct_massage', con=con, method='multi', if_exists='append', index=False)
        print(f'Data baru sejumlah {data.shape[0]}')
    else: print("No Massage")
    sleep(60)

No Massage
No Massage
list index out of range -response
list index out of range -response
[['Hai kakak, panggil aku Chass, apakah ada yang bisa saya bantu?', 0, 0], ['Maaf Kak Saya Tidak Mengerti(from respons)', False, None]]
Data baru sejumlah 2
list index out of range -response
[['Hai kakak!', 0, 0]]
Data baru sejumlah 1
list index out of range -response
[['Halo kak! mau ganti ATM yaaa, boleh banget sini Chass bantu hehehe. Oke kaka bisa banget ganti kartu untuk kartu ATM rusak, kartu ATM tertelan, dan kartu ATM hilang dengan mengujungi lokasi kantor cabang terdekat atau sama aku nanti aku bantu sepenuhnya, namun akan dikenakan biaya tambahan untuk materai 10000 sebesar Rp10.000 pembayarannya akan otomatis dipotong melalui kartu bank untuk penggantian kartu lama dengan kartu yang baru, kakak cukup ketik #gantiATM untuk melanjutkan proses penggantiannya.', 1, 0]]
Data baru sejumlah 1
No Massage
list index out of range -response
[['Silahkan kunjungi website resmi kami di www.bankhackti

TweepyException: Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Max retries exceeded with url: /1.1/direct_messages/events/list.json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001814B804070>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))